### Урок 3. Инструменты работы и визуализации ч.2

Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame (пример данных ниже).

In [1]:
data = [
    (1, "2023-11-20", "Electronics", 100, 12000),
    (2, "2023-11-21", "Electronics", 110, 13000),
    (3, "2023-11-22", "Electronics", 105, 12500),
    (4, "2023-11-20", "Clothing", 300, 15000),
    (5, "2023-11-21", "Clothing", 280, 14000),
    (6, "2023-11-22", "Clothing", 320, 16000),
    (7, "2023-11-20", "Books", 150, 9000),
    (8, "2023-11-21", "Books", 200, 12000),
    (9, "2023-11-22", "Books", 180, 10000)
]
columns = ["id", "date", "category", "quantity", "revenue"]

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import (avg, col, rank, row_number, sum, round, dense_rank)
from pyspark.sql.types import DateType, IntegerType, StringType

spark = SparkSession.builder.appName("WindowFunctionExample").getOrCreate()

In [11]:
# Создание DataFrame
df = spark.createDataFrame(data, columns)

In [12]:
df.show()

+---+----------+-----------+--------+-------+
| id|      date|   category|quantity|revenue|
+---+----------+-----------+--------+-------+
|  1|2023-11-20|Electronics|     100|  12000|
|  2|2023-11-21|Electronics|     110|  13000|
|  3|2023-11-22|Electronics|     105|  12500|
|  4|2023-11-20|   Clothing|     300|  15000|
|  5|2023-11-21|   Clothing|     280|  14000|
|  6|2023-11-22|   Clothing|     320|  16000|
|  7|2023-11-20|      Books|     150|   9000|
|  8|2023-11-21|      Books|     200|  12000|
|  9|2023-11-22|      Books|     180|  10000|
+---+----------+-----------+--------+-------+



In [13]:
# Преобразование типа дынных в столбце 'date'
df = df.withColumn("date", col("date").cast(DateType()))
#df = df.withColumn("category", col("category").cast('string'))

In [14]:
# mean каждой категории продукта
w = df.withColumn('row_number',  row_number() \
                  .over(Window.partitionBy('category') \
                  .orderBy('date')))

w.show()

+---+----------+-----------+--------+-------+----------+
| id|      date|   category|quantity|revenue|row_number|
+---+----------+-----------+--------+-------+----------+
|  7|2023-11-20|      Books|     150|   9000|         1|
|  8|2023-11-21|      Books|     200|  12000|         2|
|  9|2023-11-22|      Books|     180|  10000|         3|
|  4|2023-11-20|   Clothing|     300|  15000|         1|
|  5|2023-11-21|   Clothing|     280|  14000|         2|
|  6|2023-11-22|   Clothing|     320|  16000|         3|
|  1|2023-11-20|Electronics|     100|  12000|         1|
|  2|2023-11-21|Electronics|     110|  13000|         2|
|  3|2023-11-22|Electronics|     105|  12500|         3|
+---+----------+-----------+--------+-------+----------+



In [15]:
w.groupBy('category').agg(round(avg("revenue"), 2).alias("avg_revenue")).show()

+-----------+-----------+
|   category|avg_revenue|
+-----------+-----------+
|Electronics|    12500.0|
|   Clothing|    15000.0|
|      Books|   10333.33|
+-----------+-----------+



In [16]:
# Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, 
# в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день.
pivot_df = w.groupBy('category').pivot("date").agg(avg("revenue"))

In [17]:
pivot_df.show()

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|Electronics|   12000.0|   13000.0|   12500.0|
|   Clothing|   15000.0|   14000.0|   16000.0|
|      Books|    9000.0|   12000.0|   10000.0|
+-----------+----------+----------+----------+

